In [4]:


import numpy as np  
import pandas as pd  

from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

import re


In [6]:
data = pd.read_csv('./Sentiment.csv')

data = data[['text','sentiment']]
data


,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
...,...,...
13866,RT @cappy_yarbrough: Love to see men who will ...,Negative
13867,RT @georgehenryw: Who thought Huckabee exceede...,Positive
13868,"RT @Lrihendry: #TedCruz As President, I will a...",Positive
13869,RT @JRehling: #GOPDebate Donald Trump says tha...,Negative


In [8]:
data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)


<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_17097/1367253385.py:3: SyntaxWarning: invalid escape sequence '\s'
  data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9 \s]','',x)))
/tmp/ipykernel_17097/1367253385.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[0] = row[0].replace('rt',' ')
/tmp/ipykernel_17097/1367253385.py:9: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  row[0] = row[0].replace('rt',' ')


4472
16986


In [12]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.summary()


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_3             │ ?                      │   0 (unbuilt) │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [14]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)


(7188, 28) (7188, 2)
(3541, 28) (3541, 2)


In [15]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)


Epoch 1/7
225/225 - 10s - 44ms/step - accuracy: 0.8122 - loss: 0.4484
Epoch 2/7
225/225 - 6s - 29ms/step - accuracy: 0.8639 - loss: 0.3262
Epoch 3/7
225/225 - 7s - 30ms/step - accuracy: 0.8808 - loss: 0.2874
Epoch 4/7
225/225 - 7s - 29ms/step - accuracy: 0.8957 - loss: 0.2566
Epoch 5/7
225/225 - 6s - 28ms/step - accuracy: 0.9046 - loss: 0.2319
Epoch 6/7
225/225 - 7s - 32ms/step - accuracy: 0.9161 - loss: 0.2105
Epoch 7/7
225/225 - 7s - 30ms/step - accuracy: 0.9239 - loss: 0.1907


In [17]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))


17/17 - 0s - 10ms/step - accuracy: 0.8170 - loss: 0.5001
score: 0.50
acc: 0.82


In [18]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")


1/1 - 1s - 866ms/step
1/1 - 0s - 20ms/step
1/1 - 0s - 16ms/step
1/1 - 0s - 16ms/step
1/1 - 0s - 19ms/step
1/1 - 0s - 18ms/step
1/1 - 0s - 16ms/step
1/1 - 0s - 18ms/step
1/1 - 0s - 16ms/step
1/1 - 0s - 18ms/step
1/1 - 0s - 17ms/step
1/1 - 0s - 20ms/step
1/1 - 0s - 20ms/step
1/1 - 0s - 25ms/step
1/1 - 0s - 18ms/step
1/1 - 0s - 21ms/step
1/1 - 0s - 20ms/step
1/1 - 0s - 18ms/step
1/1 - 0s - 16ms/step
1/1 - 0s - 18ms/step
1/1 - 0s - 20ms/step
1/1 - 0s - 17ms/step
1/1 - 0s - 19ms/step
1/1 - 0s - 19ms/step
1/1 - 0s - 18ms/step
1/1 - 0s - 18ms/step
1/1 - 0s - 17ms/step
1/1 - 0s - 17ms/step
1/1 - 0s - 17ms/step
1/1 - 0s - 19ms/step
1/1 - 0s - 19ms/step
1/1 - 0s - 18ms/step
1/1 - 0s - 20ms/step
1/1 - 0s - 18ms/step
1/1 - 0s - 18ms/step
1/1 - 0s - 18ms/step
1/1 - 0s - 20ms/step
1/1 - 0s - 20ms/step
1/1 - 0s - 17ms/step
1/1 - 0s - 16ms/step
1/1 - 0s - 28ms/step
1/1 - 0s - 17ms/step
1/1 - 0s - 19ms/step
1/1 - 0s - 20ms/step
1/1 - 0s - 20ms/step
1/1 - 0s - 19ms/step
1/1 - 0s - 17ms/step
1/1 - 0s - 1

In [19]:
twt = ['Meetings: Because none of us is as dumb as all of us.']

twt = tokenizer.texts_to_sequences(twt)

twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")


[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0  204  625    7  152    6   56 1039   56   47    7  152]]
1/1 - 0s - 31ms/step
negative
